## Nested Queries: Existential and Universal Quanitification

`A op ANY <nested query>` is satisfied if **there is** ($\exists$) a value X in the result of the `<nested query>` and the condition `A op X` is satisfied
- ANY aka SOME

`A op ALL <nested query>` is satisfied if **for every** ($\forall$) a value X in the result of the `<nested query>` the condition `A op X` is satisfied

EX: Find directors of currently playing movies

(not a particularly good implementation, just to showcase the ANY keyword)

```sql
SELECT	Director
FROM	Movie
WHERE	Title = ANY     -- Syntactic sugar, equivalent to checking membership with IN keyword
    SELECT  Title
    FROM    Schedue
```

EX: Find employees w/ the highest salary
```sql
SELECT	Name
FROM	Employee
WHERE	Salary >= ALL   -- not syntactic sugar
    SELECT  Salary
    FROM    Employee
```

## Nested Queries: Set Comparisons

`<nested query 1> CONTAINS <nested query 2>`

checks if any first query results are members of the results of the second query

> The original SQL as specified for SYSTEM R  had a CONTAINS operator. This was __dropped from the language__ possibly because of the difficulty in implementing it efficiently

## Nested Queries in FROM Clause

- SQL allows nested queries in the FROM clause

EX: Find directors of movies showing in Hillcrest

```sql
SELECT	m.director
FROM	movie m, (  SELECT  title
                    FROM    schedule
                    WHERE   theater = "Hillcrest") t
WHERE   m.title = t.title
```

> Note: This is syntactic sugar and can be eliminated. You don't have to and you prob don't want to, it's just to show that you can.

# Null values in SQL

Testing if attribute is null:
- `A IS NULL`
- `A IS NOT NULL`

Ex: Find all employees with unknown phone number

```sql
SELECT	name
FROM	employee
WHERE	phone IS NULL
```

Arithmetic operations involving any NULL returns NULL

If `Salary IS NULL`, then `Salary + 1 => NULL`

Comparisons involving null return UNKNOWN (**new truth value!**)

If `Salary IS NULL`, then `Salary = 0 => UNKNOWN`


### Brief digresssion into SQL Booleans

NULL comparisons returning UNKNOWN introduces a 3rd truth value, updating the truth table like so:

![](images/2022-10-25-15-52-55.png)

WHERE clause conditions evaluating to UNKNOWN are treated as FALSE

EX: Various queries working with the following data

```
Movie   | title     director    actor
        | Tango     Berto       Brando
        | Psycho    Hitch       Perkins
        | Bambi     null        null
```

---

`SELECT title WHERE dir = 'Hitch'` => Psycho

`SELECT title WHERE dir <> 'Hitch'` => Tango
- Comparing 'Hitch' to IDK => IDK. Which is treated as FALSE (Bambi is excluded)

---

`SELECT title WHERE dir = 'null'` => EMPTY
- Comparing IDK to 'null' => IDK. Which is treated as FALSE (Bambi is excluded)

`SELECT title WHERE dir IS NULL` => Bambi

### Anomolies of null semantics

if Salary is NULL, then

-- Salary > 0 evalutes to UNKNOWN even if the domain is restricted to positive integers in schema defn

-- Consider the queries

```sql
SELECT  name
FROM    employee
WHERE   Salary <= 100 OR > 100
```

and 

```sql
SELECT  name
FROM    employee
```

These are NOT equivalent if some salaries are NULL

### Null Values and Aggregates

Total all loan ammounts

```sql
SELECT  SUM(amount)
FROM    loan
```

^ will ignore NULL ammounts. Result is NULL if there are no non-NULL amounts 

- All aggregate operations except `COUNT(*)` ignore tuples with NULL values on the aggregated attributes

Considering `R`, with a single attribute `A`,

`SELECT COUNT(*) FROM R` vs `SELECT COUNT(A) FROM R`

are NOT equivalent if `R` contains any NULL values (in which case `COUNT(*) > COUNT(A)`)

### Null Values and Group-By

NULL GROUP-BY attributes are treated like any other value

![](images/2022-10-25-16-22-08.png)

> note: Unlike most WHERE clause consider UNKNOWNS as FALSE and will thus exclude any comparisons against NULL valued tuples, the GROUP-BY will still group them with the implicit equality tests - and those evaluations (not FALSE) are inconsistent with the expliciti equality tests (FALSE). Prof particularly dislikes this inconsistency and believes it to be a fault of the group creating the standards

### Creating NULLs with OUTER JOINs

Idea: To avoid losing tuples in natural joins (recall Bambi), pad with null values

`P <outer join> Q`

> note: not needed for hw, this is all syntactic sugar. No additional expressive power, just for convenience.

Variations
- Natural Left: keep all tuples from left relation (P) (even if no join partner in/from Q)
- Natural Right keep all tuples from right relation (Q) (even if no join partner in/from P)
- Natural Full: keep all tuples from both relations

![](images/2022-10-25-16-29-19.png)

VS inner join, Movie's "Sky" tuple would've been excluded due to not having a join partner in Schedule